In [1]:
// import everything and define a test runner function
var helper = require('./helper')
var ecc = require('./ecc')
var BN = require('bn.js')
const clearModule = require('clear-module')

In [4]:
console.log('\nSigning Example')
G = ecc.G;
N = ecc.N;

secret = new BN('1800555555518005555555',10)
z = Buffer.from(helper.doubleSha256(Buffer.from('ECDSA is awesome!')),'hex')
z = new BN(z)
randMax = new BN('115792089237316195423570985008687907853269984665640564039457584007913129639936',10)
k = randMax.muln(Math.random())
r = G.rmul(k).x.num
s = r.mul(secret).add(z)

var red = BN.red(N)
var redk = k.toRed(red)

s = (r.mul(secret).add(z)).mul(redk.redPow(N.subn(2)).mod(N)).mod(N)
console.log(z.toString(16))
console.log(r.toString(16))
console.log(s.toString(16))



Signing Example
<BN: fffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141>
cf6304e0ed625dc13713ad8b330ca764325f013fe7a3057dbe6a2053135abeb4
35d7466ea41dbb145f743d58355b7eab0b64381de0230c594ad25817347af44f
a4c170c2280c1721dc343b28dfdf0653b509e4b632c8afd40d4e77501cf859fc


In [5]:
//Verification Example
console.log('\nVerification Example')
point = new ecc.S256Point(
			new BN('04519fac3d910ca7e7138f7013706f619fa8f033e6ec6e09370ea38cee6a7574', 16),
			new BN('82b51eab8c27c66e26c858a079bcdf4f1ada34cec420cafc7eac1a42216fb6c4', 16));
z = new BN('bc62d4b80d9e36da29c16c5d4d9f11731f36052c72401a76c23c0fb5a9b74423', 16);
r = new BN('37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6', 16);
s = new BN('8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec', 16);

var red = BN.red(N)
var reds = s.toRed(red)
			
u = z.mul(reds.redPow(N.subn(2))).mod(N)
v = r.mul(reds.redPow(N.subn(2))).mod(N)

console.log(G.rmul(u).add(point.rmul(v)).x.num.eq(r))



Verification Example
true


### Exercise 1

#### 1.1. Which sigs are valid?

```
P = (887387e452b8eacc4acfde10d9aaf7f6d9a0f975aabb10d006e4da568744d06c, 
     61de6d95231cd89026e286df3b6ae4a894a3378e393e93a0f45b666329a0ae34)
z, r, s = ec208baa0fc1c19f708a9ca96fdeff3ac3f230bb4a7ba4aede4942ad003c0f60,
          ac8d1c87e51d0d441be8b3dd5b05c8795b48875dffe00b7ffcfac23010d3a395,
          68342ceff8935ededd102dd876ffd6ba72d6a427a3edb13d26eb0781cb423c4
z, r, s = 7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d,
          eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c,
          c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6
```

#### 1.2. Make [these tests](/edit/session3/ecc.py) pass
```
ecc.py:S256Test:test_verify
ecc.py:PrivateKeyTest:test_sign
```

In [6]:
console.log('\nExercise 1.1')

px = '887387e452b8eacc4acfde10d9aaf7f6d9a0f975aabb10d006e4da568744d06c'
py = '61de6d95231cd89026e286df3b6ae4a894a3378e393e93a0f45b666329a0ae34'

signatures = [
       ['ec208baa0fc1c19f708a9ca96fdeff3ac3f230bb4a7ba4aede4942ad003c0f60',
		'ac8d1c87e51d0d441be8b3dd5b05c8795b48875dffe00b7ffcfac23010d3a395',
		'68342ceff8935ededd102dd876ffd6ba72d6a427a3edb13d26eb0781cb423c4'],
	   ['7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d',
		'eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c',
		'c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6'],
   ]
point = new ecc.S256Point(new BN(px,16), new BN(py,16))
signatures.map(obj => {
	var red = BN.red(N)
	z = new BN(obj[0],16)
	r = new BN(obj[1],16)
	s = new BN(obj[2],16)
	var reds = s.toRed(red)
	u = z.mul(reds.redPow(N.subn(2))).mod(N)
	v = r.mul(reds.redPow(N.subn(2))).mod(N)
	console.log(G.rmul(u).add(point.rmul(v)).x.num.eq(r))

})



Exercise 1.1
true
true


[ undefined, undefined ]

In [10]:
//# Exercise 1.2

clearModule.all();
helper.runTest('test_verify');
clearModule.all();

helper.runTest('test_sign');





  S256Test
  PrivateKeyTest
    ✓ test_sign (482ms)

    ✓ test_verify (514ms)


  1 passing (1s)


  1 passing (1s)



### Exercise 2

#### 2.1. Verify the DER signature for the hash of "ECDSA is awesome!" for the given SEC pubkey

`z = int.from_bytes(double_sha256('ECDSA is awesome!'), 'big')`

Public Key in SEC Format: 
0204519fac3d910ca7e7138f7013706f619fa8f033e6ec6e09370ea38cee6a7574

Signature in DER Format: 304402201f62993ee03fca342fcb45929993fa6ee885e00ddad8de154f268d98f083991402201e1ca12ad140c04e0e022c38f7ce31da426b8009d02832f0b44f39a6b178b7a1


In [11]:
console.log('\nExercise 2.1')

der = Buffer.from('304402201f62993ee03fca342fcb45929993fa6ee885e00ddad8de154f268d98f083991402201e1ca12ad140c04e0e022c38f7ce31da426b8009d02832f0b44f39a6b178b7a1','hex')
sec = Buffer.from('0204519fac3d910ca7e7138f7013706f619fa8f033e6ec6e09370ea38cee6a7574','hex')
z = new BN(helper.doubleSha256(Buffer.from('ECDSA is awesome!')),'hex')

sig = ecc.Signature.parse(der)
point = ecc.S256Point.parse(sec)
console.log(point.verify(z,sig))



Exercise 2.1
93803711187162121691076966012382940511580455351029572768930182870238504861364
true


### Exercise 3

WIF is the serialization of a Private Key.

#### 3.1. Find the WIF Format of the following:

* \\(2^{256}-2^{199}\\), mainnet, compressed
* \\(2^{256}-2^{201}\\), testnet, uncompressed
* 0dba685b4511dbd3d368e5c4358a1277de9486447af7b3604a69b8d9d8b7889d, mainnet, uncompressed
* 1cca23de92fd1862fb5b76e5f4f50eb082165e5191e116c18ed1a6b24be6a53f, testnet, compressed

#### 3.2. Make [this test](/edit/session3/ecc.py) pass
```
ecc.py:PrivateKeyTest:test_wif
```

In [12]:
// WIF Example
console.log('\nWIF Example')
secret1 = Buffer.from('115792089237316193816632940749697632311307892324477961517254590225120294338560')
secret2 = Buffer.from('ffffffffffffff00000000000000000000000000000000000000000000000000', 'hex')
x = new BN('115792089237316193816632940749697632311307892324477961517254590225120294338560',10)
y = new BN('ffffffffffffff00000000000000000000000000000000000000000000000000', 16)
xb = x.toBuffer('be')
yb = y.toBuffer('be')
console.log(helper.encodeBase58Checksum(Buffer.concat([Buffer.from([0x80]), secret2])))
console.log(helper.encodeBase58Checksum(Buffer.concat([Buffer.from([0x80]), secret2, Buffer.from([0x01])])))
console.log(helper.encodeBase58Checksum(Buffer.concat([Buffer.from([0xef]), secret2])))
console.log(helper.encodeBase58Checksum(Buffer.concat([Buffer.from([0xef]), secret2, Buffer.from([0x01])])))



WIF Example
5Km2kuu7vtF1mmnTprG5Qtchx5NYEBHp9X1nm6v1dNJSND1E5Ge
L5oLkpV3aqArQcffQCRYUSpbk6J4qARD6bUUyf6ymX1xxX9iVxpK
93XfLeifX7K9jqHkTC9zHVAfbjjFPLq1VTsjqjGWy73V9JUWGuo
cWALDjUu1ts7a48vncEfqmKfNKbUVcWuAdcx65ZVGdfyDGDYyszf


In [13]:
// Exercise 3.1
console.log('\nExercise 3.1')
components = [
				['ffffffffffffff80000000000000000000000000000000000000000000000000', false, true],
				['fffffffffffffe00000000000000000000000000000000000000000000000000', true,  false],
				['0dba685b4511dbd3d368e5c4358a1277de9486447af7b3604a69b8d9d8b7889d', false, false],
				['1cca23de92fd1862fb5b76e5f4f50eb082165e5191e116c18ed1a6b24be6a53f', true, true]
]

components.map( obj => {
	secretBytes = Buffer.from(obj[0],'hex')
	if (obj[1]) {
		prefix = Buffer.from([0xef])
	} else {
		prefix = Buffer.from([0x80])
	}
	if (obj[2]) {
		suffix = Buffer.from([0x01])
	} else {
		suffix = Buffer.from([])
	}
	wif = helper.encodeBase58Checksum(Buffer.concat([prefix, secretBytes, suffix]))
	console.log(wif)
})


Exercise 3.1
L5oLkpV3aqBJ4BgssVAsax1iRa77G5CVYnv9adQ6Z87te7TyUdSC
93XfLeifX7Jx7n7ELGMAf1SUR6f9kgQs8Xke8WStMwUtrDucMzn
5HvLFPDVgFZRK9cd4C5jcWki5Skz6fmKqi1GQJf5ZoMofid2Dty
cNYfWuhDpbNM1JWc3c6JTrtrFVxU4AGhUKgw5f93NP2QaBqmxKkg


[ undefined, undefined, undefined, undefined ]

In [15]:
//# Exercise 3.2

clearModule.all();

helper.runTest('test_wif');



  PrivateKeyTest
    ✓ test_wif (192ms)


  1 passing (194ms)



### Exercise 4

#### 4.1. Make [this test](/edit/session3/tx.py) pass
```
tx.py:TxTest:test_parse_version
```

In [47]:
buf = Buffer.from([0x12, 0x34, 0x56, 0x78, 0x90, 0xab, 0x12, 0x34]);
function mapReverse(array, fn) {
    return array.reduceRight(function (result, el) {
        result.push(fn(el));
        return result;
    }, []);
}

mapReverse(buf, x=>{
    //console.log(x.toString(16))
    x
})

buf = Buffer.from('a135ef0100000000','hex')
uar = Uint8Array.from(buf)
console.log(new BN(buf.reverse().toString('hex'),'hex').toNumber(10))

32454049


In [17]:
//# Exercise 4.1

clearModule.all()
helper.runTest('test_parse_version')

RangeError [ERR_OUT_OF_RANGE]: The value of "byteLength" is out of range. It must be >= 1 and <= 6. Received 8

### Exercise 5

#### 5.1. Make [this test](/edit/session3/tx.py) pass
```
tx.py:TxTest:test_parse_inputs
```

In [ ]:
# Exercise 5.1

reload(tx)
run_test(tx.TxTest('test_parse_inputs'))

### Exercise 6

#### 6.1. Make [this test](/edit/session3/tx.py) pass
```
tx.py:TxTest:test_parse_outputs
```

In [ ]:
# Exercise 6.1

reload(tx)
run_test(tx.TxTest('test_parse_outputs'))

### Exercise 7

#### 7.1. Make [this test](/edit/session3/tx.py) pass
```
tx.py:TxTest:test_parse_locktime
```

In [ ]:
# Exercise 7.1

reload(tx)
run_test(tx.TxTest('test_parse_locktime'))

### Exercise 8

#### 8.1. What is the scriptSig from the second input in this tx?
#### 8.2. What is the scriptPubKey and amount of the first output in this tx?
#### 8.3. What is the amount for the second output?

```
010000000456919960ac691763688d3d3bcea9ad6ecaf875df5339e148a1fc61c6ed7a069e010000006a47304402204585bcdef85e6b1c6af5c2669d4830ff86e42dd205c0e089bc2a821657e951c002201024a10366077f87d6bce1f7100ad8cfa8a064b39d4e8fe4ea13a7b71aa8180f012102f0da57e85eec2934a82a585ea337ce2f4998b50ae699dd79f5880e253dafafb7feffffffeb8f51f4038dc17e6313cf831d4f02281c2a468bde0fafd37f1bf882729e7fd3000000006a47304402207899531a52d59a6de200179928ca900254a36b8dff8bb75f5f5d71b1cdc26125022008b422690b8461cb52c3cc30330b23d574351872b7c361e9aae3649071c1a7160121035d5c93d9ac96881f19ba1f686f15f009ded7c62efe85a872e6a19b43c15a2937feffffff567bf40595119d1bb8a3037c356efd56170b64cbcc160fb028fa10704b45d775000000006a47304402204c7c7818424c7f7911da6cddc59655a70af1cb5eaf17c69dadbfc74ffa0b662f02207599e08bc8023693ad4e9527dc42c34210f7a7d1d1ddfc8492b654a11e7620a0012102158b46fbdff65d0172b7989aec8850aa0dae49abfb84c81ae6e5b251a58ace5cfeffffffd63a5e6c16e620f86f375925b21cabaf736c779f88fd04dcad51d26690f7f345010000006a47304402200633ea0d3314bea0d95b3cd8dadb2ef79ea8331ffe1e61f762c0f6daea0fabde022029f23b3e9c30f080446150b23852028751635dcee2be669c2a1686a4b5edf304012103ffd6f4a67e94aba353a00882e563ff2722eb4cff0ad6006e86ee20dfe7520d55feffffff0251430f00000000001976a914ab0c0b2e98b1ab6dbf67d4750b0a56244948a87988ac005a6202000000001976a9143c82d7df364eb6c75be8c80df2b3eda8db57397088ac46430600
```

In [ ]:
# Exercise 8.1/8.2/8.3

from io import BytesIO
from tx import Tx

hex_transaction = '010000000456919960ac691763688d3d3bcea9ad6ecaf875df5339e148a1fc61c6ed7a069e010000006a47304402204585bcdef85e6b1c6af5c2669d4830ff86e42dd205c0e089bc2a821657e951c002201024a10366077f87d6bce1f7100ad8cfa8a064b39d4e8fe4ea13a7b71aa8180f012102f0da57e85eec2934a82a585ea337ce2f4998b50ae699dd79f5880e253dafafb7feffffffeb8f51f4038dc17e6313cf831d4f02281c2a468bde0fafd37f1bf882729e7fd3000000006a47304402207899531a52d59a6de200179928ca900254a36b8dff8bb75f5f5d71b1cdc26125022008b422690b8461cb52c3cc30330b23d574351872b7c361e9aae3649071c1a7160121035d5c93d9ac96881f19ba1f686f15f009ded7c62efe85a872e6a19b43c15a2937feffffff567bf40595119d1bb8a3037c356efd56170b64cbcc160fb028fa10704b45d775000000006a47304402204c7c7818424c7f7911da6cddc59655a70af1cb5eaf17c69dadbfc74ffa0b662f02207599e08bc8023693ad4e9527dc42c34210f7a7d1d1ddfc8492b654a11e7620a0012102158b46fbdff65d0172b7989aec8850aa0dae49abfb84c81ae6e5b251a58ace5cfeffffffd63a5e6c16e620f86f375925b21cabaf736c779f88fd04dcad51d26690f7f345010000006a47304402200633ea0d3314bea0d95b3cd8dadb2ef79ea8331ffe1e61f762c0f6daea0fabde022029f23b3e9c30f080446150b23852028751635dcee2be669c2a1686a4b5edf304012103ffd6f4a67e94aba353a00882e563ff2722eb4cff0ad6006e86ee20dfe7520d55feffffff0251430f00000000001976a914ab0c0b2e98b1ab6dbf67d4750b0a56244948a87988ac005a6202000000001976a9143c82d7df364eb6c75be8c80df2b3eda8db57397088ac46430600'

# bytes.fromhex to get the binary representation
# create a stream using BytesIO()
# Tx.parse() the stream
# print tx's second input's scriptSig
# print tx's first output's scriptPubKey
# print tx's first output's amount